### 2017-2021 버스 데이터 전처리 
---
- 년도별 반복작업
- 데이터 중 columns 명이 다르면 따로 수정
- 2020년 데이터 중 columns 명이 다르거나 위치가 다르게 되어 있는 부분은 직접 수정함.

In [2]:
import pandas as pd
import glob
import json
import requests

In [ ]:
# csv 파일을 불러 병합
csvfiles = glob.glob('data/*.csv')
data = pd.DataFrame()

for csvfile in csvfiles:
    tmp = pd.read_csv(csvfile, encoding='cp949', low_memory = False)
    data = pd.concat([data, tmp])
    
data.reset_index(drop=True, inplace=True)

In [ ]:
df2017_sum = data.drop(columns=['노선번호', '노선명','역명','승차총승객수', '등록일자'])

In [ ]:
df2017_sum=df2021_sum.groupby(['사용일자', '버스정류장ARS번호'])['하차총승객수'].sum().reset_index()

In [ ]:
df2017_sum['버스정류장ARS번호']=df2017_sum['버스정류장ARS번호'].str.replace('~','').replace('',0).astype('int')

In [ ]:
data =pd.read_csv('data/20220429기준_서울시정류소리스트.csv', encoding='cp949')

In [ ]:
data = data.drop(columns=['NODE_ID','정류소명'])

In [ ]:
data['버스정류장ARS번호']=data['버스정류장ARS번호'].astype('int')

In [ ]:
bus2017=  pd.merge(left = df2017_sum, right = data, on = '버스정류장ARS번호', how = 'left')

In [ ]:
bus2017= bus2017.dropna()

In [ ]:
bus2021['년'] = bus2021['사용일자'].astype('string').str[:4].astype('int')
bus2021['월'] = bus2021['사용일자'].astype('string').str[4:6].astype('int')
bus2021['일'] = bus2021['사용일자'].astype('string').str[6:].astype('int')

In [ ]:
bus2017 = bus2017[['사용일자', '년', '월', '일', '버스정류장ARS번호', '하차총승객수', 'X좌표', 'Y좌표']]

In [ ]:
api_key = '암호'

In [ ]:
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address_name']
    return addr.append(match_first)

In [ ]:
addr = []
for i in range(0,2):
    try: lat_lon_to_addr(data['좌표X'][i],data['좌표Y'][i])
    except : 
        addr.append(None)

In [ ]:
addr = pd.DataFrame(addr)

In [ ]:
addr['버스정류장ARS번호'] = bus2017['버스정류장ARS번호']
addr.rename(columns={0:'주소'},inplace=True)
addr=addr[['버스정류장ARS번호', '주소']]

In [ ]:
#병합하고 분리
bus2017 = pd.merge(bus2017,addr, on = '버스정류장ARS번호', how='left')

bus2017['구'] = bus2017['주소'].str.split(' ').str.get(1)
bus2017['동'] = bus2017['주소'].str.split(' ').str.get(2)

# 저장해서 로드 시간을 줄입니다....
bus2017.to_csv('2017버스.csv', index=False)

### 데이터 시각화
---

- Calendar : 일별 하차승객 데이터 

In [ ]:
pip install pyecharts

In [ ]:
import pyecharts
from pyecharts.charts import Calendar
from pyecharts import options as opts

In [ ]:
bus2017['사용일자']= bus2017['사용일자'].astype('str')

In [ ]:
bus2017['사용일자'] = pd.to_datetime(bus2017['사용일자'])

In [ ]:
day_by_year = bus2017.groupby('사용일자').sum()[['하차총승객수']].round(0)

In [ ]:
day_by_year = day_by_year.reset_index()
day_by_year

In [ ]:
data = day_by_year[['사용일자','하차총승객수']].values.tolist()
data

In [ ]:
max_day = day_by_year['하차총승객수'].max()
min_day = day_by_year['하차총승객수'].min()

In [ ]:
cal = (Calendar()
 .add('',data,calendar_opts=opts.CalendarOpts(range_='2019'))
 .set_global_opts(
     title_opts=opts.TitleOpts(title='2019년도 버스', subtitle='일별 데이터'),
     legend_opts=opts.LegendOpts(is_show=False),
     visualmap_opts=opts.VisualMapOpts(
         max_ = max_day,
         min_ = min_day,
         orient='horizontal',
         is_piecewise=False,
         pos_top='230px',
         pos_left='100px',
     )))

In [4]:
cal.render() #생성된 파일 확인

NameError: name 'cal' is not defined

- 지도 HeatMap

In [ ]:
import folium

In [ ]:
seoul_loc = (37.566535, 126.9779692)
seoul = folium.Map(seoul_loc, zoom_start=10, tiles='stamenterrain')
seoul.add_child(plugins.HeatMap(zip(year170929['Y좌표'], 
                                    year170929['X좌표'], 
                                    year170929['하차총승객수']),
                                radius=10, min_opacity=0.1))

- 피벗 테이블

In [ ]:
bb17= bus2017.groupby(['월','일','동'])['하차총승객수'].sum().reset_index()
bb17

In [ ]:
b17 = bus2017.pivot_table(index=['월','일'], columns='동', values = '하차총승객수')
b17

- Line 연도별 하차승객수

In [5]:
pip install cufflinks
pip install chart_studio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [ ]:
mon2017 = bus2017.groupby('월')['하차총승객수'].sum().reset_index()
mon2018 = bus2018.groupby('월')['하차총승객수'].sum().reset_index()
mon2019 = bus2019.groupby('월')['하차총승객수'].sum().reset_index()
mon2020 = bus2020.groupby('월')['하차총승객수'].sum().reset_index()
mon2021 = bus2021.groupby('월')['하차총승객수'].sum().reset_index()

In [ ]:
monAll=pd.concat([mon2017,mon2018,mon2019,mon2020,mon2021], axis=1)

In [ ]:
monAll.columns = ['월', '2017승객수', 'ㅇ월', '2018승객수', 'ㄹ월', '2019승객수', 'ㅎ월', '2020승객수', 'ㅗ월',
       '2021승객수']

In [ ]:
monAll = monAll[['월','2017승객수','2018승객수','2019승객수','2020승객수','2021승객수']]

In [ ]:
monAll.iplot(kind='line', title='2017년-2021년 월별 유동인구',x = '월')

- Line 월별 하차승객수

In [ ]:
bus2017.groupby('월')['하차총승객수'].sum().iplot(kind='line', theme = 'solar')

- Bar

In [ ]:
yearAll=pd.concat([bus2017.groupby('년')['하차총승객수'].sum().reset_index(),
           bus2018.groupby('년')['하차총승객수'].sum().reset_index(),
           bus2019.groupby('년')['하차총승객수'].sum().reset_index(),
           bus2020.groupby('년')['하차총승객수'].sum().reset_index(),
           bus2021.groupby('년')['하차총승객수'].sum().reset_index()])

In [ ]:
yearAll.reset_index(drop=True, inplace = True)

In [ ]:
yearAll['년'].astype('string')

In [ ]:
yearAll.iplot(kind='bar', x= '년', y='하차총승객수', theme='ggplot' ,title='2017년-2021년 년도별 유동인구' )